# Cookbook: Using PDB files with the OpenFF Toolkit

PDB files are a common way to represent biopolymer structures, but they don't natively contain all of the chemical information required to construct a `Molecule` object(s). However, PDB files are used widely in the molecular simulation community, so OpenFF has implemented functionality to load them.

The recommended pathway for loading PDB files is [`Topology.from_pdb`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology.from_pdb). Additionally, the PDB file must have the following characteristics:

For protein atoms:
* Atom names and residue names must be consistent with the [Chemical Components Dictonary](https://www.wwpdb.org/data/ccd)
* Only the 20 canonical amino acids are supported by default (including protonated/deprotonated variants)
* All hydrogens must be explicit

For small molecules, waters, and ions:
* The element of each atom must be identified in the final column of the file
* Bonds must be identified in the CONECT records at the bottom of the file
* All unique small molecules must be identified in the `unique_molecules` keyword argument

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ℹ️ OpenFF <code>Topology</code> and <code>Molecule</code> objects store much more information than is in the PDB files they might be generated from. Don't be surprised if this Python code takes a few more seconds to load a PDB file than other tools.
</div>

In [1]:
from openff.toolkit import Topology
from viz import visualize_topology

## PDB file with only a (single) protein

`6hvi_prepared.pdb` is a prepared protein from the [Merck free energy perturbation study](https://github.com/MCompChem/fep-benchmark/blob/master/pfkfb3/6hvi_prepared.pdb). It was prepared using commercial tools by Schrodinger, and doesn't have any (solvent) waters, ions, or small molecules.

In [2]:
protein = Topology.from_pdb("6hvi_prepared.pdb")

visualize_topology(protein)

NGLWidget()

From here, you might use Packmol or PDBFixer to add water or other solvents without leaving Python (or there are tools available for water/solvent packing in AMBER, GROMACS, and many commercial packages). An example of using PDBFixer can be found in our ["Toolkit Showcase" example](https://docs.openforcefield.org/en/latest/examples/openforcefield/openff-toolkit/toolkit_showcase/toolkit_showcase.html).

## PDB file with a protein, waters, and ions

This PDB file comes from the [AMBER tutorial on making solvated simulations](https://ambermd.org/tutorials/basic/tutorial7/index.php). It is slightly modified to create an output suitable for loading into the OpenFF Toolkit
  * TIP3P water is used instead of OPC (a four-site model)
  * a cubic solvent box is used instead of an octohedron
  * the N terminus of the protein is made neutral

In [3]:
RAMP1 = Topology.from_pdb("RAMP1_solv_box_ions.pdb")

visualize_topology(RAMP1)

NGLWidget()

# PDB file with one or more ligands

The following PDB is taken from the [GROMACS protein-ligand tutorial](http://www.mdtutorials.com/gmx/complex/index.html) (with manually-added CONECT records) and contains a protein, water, and a small molecule (ligand).

While the chemistry of water, ions, and proteins can be deduced from known templates (this is what happened under the hood in the previous two examples), small molecules are trickier. The number of amino acids and ions is relatively small, but the number of possible small molecules is tremendous so storing them in a database is not feasible. We instead ask the user to fill in the missing information that would be looked up from the CCD or hard-coded heuristics, specifically the bond order and stereochemistry. This information is stored in a `Molecule` object(s), which we ask the user to provide via the `unique_molecules` argument.

If you try to load a PDB file containing a small molecule, without providing the chemistry of the small molecule, you'll get an error message identifying the atoms that couldn't be loaded:

In [4]:
from openff.toolkit.utils.exceptions import UnassignedChemistryInPDBError

try:
    Topology.from_pdb("gromacs_solv_complex.pdb")
except UnassignedChemistryInPDBError as error:
    print(str(error))

Some bonds or atoms in the input could not be identified.

Hint: The following residue names with unassigned atoms were not found in the substructure library. While the OpenFF Toolkit identifies residues by matching chemical substructures rather than by residue name, it currently only supports the 20 'canonical' amino acids.
    JZ4


Hint: The following residues were assigned names that do not match the residue name in the input, or could not be assigned residue names at all. This may indicate that atoms are missing from the input or some other error. The OpenFF Toolkit requires all atoms, including hydrogens, to be explicit in the input to avoid ambiguities in protonation state or bond order:
    Input residue X:JZ4#0001 contains atoms matching substructures {'No match'}

Error: The following 22 atoms exist in the input but could not be assigned chemical information from the substructure library:
    Atom  2614 (C4) in residue X:JZ4#0001
    Atom  2615 (C7) in residue X:JZ4#0001
    

Any appropriate OpenFF `Molecule` object can be used to define the small molecule ligand when loading the PDB file. (See the Molecule Cookbook for some of the numerous ways to create them!).  Since the coordinates are specified in the PDB file, it's not necessary that this `Molecule` has conformers.

In this case we known the identity of the ligand and can look up a SMILES string, which is all that's needed to load this PDB file! (Since there are many molecules in this file, it may take a few tens of seconds to load - this is expected.)

In [5]:
from openff.toolkit import Molecule

jz4 = Molecule.from_smiles("CCCC1=CC=CC=C1O")
complex = Topology.from_pdb("gromacs_solv_complex.pdb", unique_molecules=[jz4])

visualize_topology(complex)

NGLWidget()